## Fine-Tuning the Gemma 3-4B Model with LoRA using UnsLoTH

Fine-tuning large language models (LLMs) is often computationally expensive and operationally complex. UnsLoTH simplifies this process by providing highly optimized training utilities, enabling efficient fine-tuning even for multi-billion-parameter models.

In this notebook, we demonstrate how to fine-tune the Gemma 3-4B model using Low-Rank Adaptation (LoRA), a parameter-efficient fine-tuning technique that significantly reduces memory usage and training cost while preserving model performance.

### Prerequisites

Before running this notebook, ensure the following requirements are met:
- Hardware: Access to a GPU-enabled machine (e.g., NVIDIA A100, V100, or equivalent).
- Software: The UnsLoTH library and its dependencies must be installed (installation steps are provided below).
- Credentials:
    - Proper authentication for Google Cloud Platform (GCP) if using a Cloud Storage bucket.
    - Optional Weights & Biases (W&B) credentials for experiment tracking.

### Important Notes
- Model checkpoints and intermediate artifacts are stored directly in a Google Cloud Storage bucket. Ensure that your GCP credentials are correctly configured and that you have access to the target bucket.
- Training metrics are logged using Weights & Biases (W&B) by default. If W&B logging is not required, it can be disabled in the training configuration.

In [ ]:
%pip install -q accelerate datasets transformers bitsandbytes
%pip install -q wandb trl unsloth

In [ ]:
# train_rag_unsloth.py
import os
import random
from dataclasses import dataclass
from typing import Dict, Any, List

from datasets import load_dataset
from transformers import (
    TrainingArguments,
    TrainerCallback,
    TrainerState,
    TrainerControl,
)
from transformers.trainer_utils import get_last_checkpoint
import numpy as np
import torch
import wandb

from unsloth import FastModel, is_bfloat16_supported
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import get_chat_template

from google.cloud import storage


# =========================
# CONFIG
# =========================
SEED = 3407
# set random seed for reproducibility

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

MODEL_NAME = "unsloth/gemma-3-4b-it"  # Base model
DATASET_NAME = "FreedomIntelligence/RAG-Instruct"  # HF dataset
OUTPUT_DIR = "./outputs/gemma3-4b-rag"

BUCKET_NAME = "model-finetune-1"  # GCP bucket for finetune checkpoints
GCS_PREFIX = "checkpoints/gemma3-4b-rag"  # optional path inside bucket
ENABLE_GCS_SYNC = True

MAX_SEQ_LENGTH = 2048  # safe for 4B on L4 with packing
MAX_DOCS = 2  # number of documents per sample
MAX_DOC_CHARS = 4000  # truncate context to avoid huge prompts

# Budget controls – tune these first
MAX_TRAIN_SAMPLES = None  # set to None for full dataset
NUM_TRAIN_EPOCHS = 1
LEARNING_RATE = 2e-4

BATCH_SIZE = 1  # per-device batch size
GRAD_ACCUM_STEPS = 4  # effective batch size = BATCH_SIZE * GRAD_ACCUM_STEPS

USE_WANDB = True  # set True and export WANDB_PROJECT if you want logging
WANDB_PROJECT = "instruct-rag-finetune"


if USE_WANDB:
    run = wandb.init(
        entity="pareek-ml-personal",
        project=WANDB_PROJECT,
        config={
            "model_name": MODEL_NAME,
            "dataset_name": DATASET_NAME,
            "max_train_samples": MAX_TRAIN_SAMPLES,
            "num_train_epochs": NUM_TRAIN_EPOCHS,
            "learning_rate": LEARNING_RATE,
            "batch_size": BATCH_SIZE,
            "grad_accum_steps": GRAD_ACCUM_STEPS,
        },
    )

/home/yashpareek_workmail/unsloth-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_17941/340567520.py:19: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastModel, is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


wandb: Currently logged in as: pareek-ml (pareek-ml-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [ ]:
# =========================
# GCS SYNC CALLBACK
# =========================


class GCSSyncCallback(TrainerCallback):
    def __init__(self, local_dir: str, bucket_name: str, prefix: str):
        self.local_dir = os.path.abspath(local_dir)
        self.bucket_name = bucket_name
        self.prefix = prefix.rstrip("/") if prefix else ""
        self.client = storage.Client()
        self.bucket = self.client.bucket(bucket_name)

    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        # Sync only the checkpoint that was just written
        if not os.path.exists(self.local_dir):
            return

        print(
            f"[GCSSync] Syncing {self.local_dir} to gs://{self.bucket_name}/{self.prefix} ..."
        )
        self._sync_directory(self.local_dir, self.prefix)
        print("[GCSSync] Sync complete.")
        return control

    def _sync_directory(self, local_dir: str, gcs_prefix: str):
        for root, dirs, files in os.walk(local_dir):
            for fname in files:
                local_path = os.path.join(root, fname)
                rel_path = os.path.relpath(local_path, local_dir)
                blob_name = f"{gcs_prefix}/{rel_path}" if gcs_prefix else rel_path
                blob = self.bucket.blob(blob_name)
                blob.upload_from_filename(local_path)

In [ ]:
# =========================
# MODEL & TOKENIZER
# =========================


def load_model_and_tokenizer():
    print("Loading base model and tokenizer...")
    model, tokenizer = FastModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        load_in_4bit=True,
        load_in_8bit=False,
        full_finetuning=False,
    )

    print("Applying LoRA (QLoRA) configuration...")
    model = FastModel.get_peft_model(
        model,
        finetune_vision_layers=False,
        finetune_language_layers=True,
        finetune_attention_modules=True,
        finetune_mlp_modules=True,
        r=8,  # LoRA rank
        lora_alpha=8,  # usually >= r
        lora_dropout=0,
        bias="none",
        random_state=3407,
    )

    # Prepare for training (gradient checkpointing etc.)
    FastModel.for_training(model)

    # Attach Gemma-3 chat template
    tokenizer = get_chat_template(
        tokenizer,
        chat_template="gemma-3",
    )

    return model, tokenizer

In [ ]:
# =========================
# DATASET + PROMPTING
# =========================


def join_docs(docs: List[str]) -> str:
    """Join top-k docs and truncate to avoid crazy-long contexts."""
    if not docs:
        return ""
    text = "\n\n".join(docs[:MAX_DOCS])
    return text[:MAX_DOC_CHARS]


def make_prompt(question: str, docs: str) -> str:
    """User prompt: question + retrieved documents."""
    return (
        "You are a helpful assistant. Use ONLY the provided documents to answer the question.\n\n"
        "QUESTION:\n"
        f"{question}\n\n"
        "DOCUMENTS:\n"
        f"{docs}\n"
    )


def formatting_single_example(example: Dict[str, Any], tokenizer) -> Dict[str, str]:
    question = example["question"]
    answer = example["answer"]
    docs = join_docs(example["documents"])

    user_prompt = make_prompt(question, docs)

    messages = [
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": answer},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )

    bos = tokenizer.bos_token or ""
    if bos and text.startswith(bos):
        text = text[len(bos) :]

    return {"text": text}


def load_and_prepare_dataset(tokenizer):
    print(f"Loading dataset: {DATASET_NAME}")
    dataset = load_dataset(DATASET_NAME, split="train")

    if MAX_TRAIN_SAMPLES is not None:
        n = min(MAX_TRAIN_SAMPLES, len(dataset))
        print(f"Subsampling dataset to {n} examples for budget.")
        dataset = dataset.select(range(n))

    def _map_fn(batch):
        questions = batch["question"]
        answers = batch["answer"]
        documents = batch["documents"]

        texts = []
        for q, a, docs in zip(questions, answers, documents):
            ex = {"question": q, "answer": a, "documents": docs}
            out = formatting_single_example(ex, tokenizer)
            texts.append(out["text"])

        return {"text": texts}

    print("Formatting dataset into chat-style text...")
    dataset = dataset.map(
        _map_fn,
        batched=True,
        remove_columns=dataset.column_names,
        desc="Formatting prompts",
    )

    return dataset

In [5]:
model, tokenizer = load_model_and_tokenizer()
dataset = load_and_prepare_dataset(tokenizer)

# Only use 80% of data for training, rest for eval
train_size = int(0.8 * len(dataset))
train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, len(dataset)))

Loading base model and tokenizer...


==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
Applying LoRA (QLoRA) configuration...
Unsloth: Making `model.base_model.model.model.language_model` require gradients
Loading dataset: FreedomIntelligence/RAG-Instruct
Formatting dataset into chat-style text...


In [8]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
bf16 = is_bfloat16_supported()
print(f"bfloat16 supported: {bf16}")
report_to = "wandb" if USE_WANDB else "none"

bfloat16 supported: True


In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    learning_rate=LEARNING_RATE,
    logging_steps=200,
    save_steps=100,
    save_total_limit=3,
    bf16=bf16,
    fp16=not bf16,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    report_to=report_to,
)

callbacks = []
if ENABLE_GCS_SYNC:
    gcs_callback = GCSSyncCallback(
        local_dir=OUTPUT_DIR,
        bucket_name=BUCKET_NAME,
        prefix=GCS_PREFIX,
    )
    callbacks.append(gcs_callback)
    print("GCS sync callback enabled.")

print("Starting SFT training...")
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=True,
    args=training_args,
    callbacks=callbacks,
)

trainer.train()

GCS sync callback enabled.
Starting SFT training...


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32,432 | Num Epochs = 1 | Total steps = 8,108
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
200,2.673700
400,2.243800
600,2.217400
800,2.194700
1000,2.191400
1200,2.165900
1400,2.148600
1600,2.168800
1800,2.145600
2000,2.154300


[GCSSync] Syncing /home/yashpareek_workmail/unsloth_finetuning/notebooks/outputs/gemma3-4b-rag to gs://model-finetune-1/checkpoints/gemma3-4b-rag ...
[GCSSync] Sync complete.
[GCSSync] Syncing /home/yashpareek_workmail/unsloth_finetuning/notebooks/outputs/gemma3-4b-rag to gs://model-finetune-1/checkpoints/gemma3-4b-rag ...
[GCSSync] Sync complete.
[GCSSync] Syncing /home/yashpareek_workmail/unsloth_finetuning/notebooks/outputs/gemma3-4b-rag to gs://model-finetune-1/checkpoints/gemma3-4b-rag ...
[GCSSync] Sync complete.
[GCSSync] Syncing /home/yashpareek_workmail/unsloth_finetuning/notebooks/outputs/gemma3-4b-rag to gs://model-finetune-1/checkpoints/gemma3-4b-rag ...
[GCSSync] Sync complete.
[GCSSync] Syncing /home/yashpareek_workmail/unsloth_finetuning/notebooks/outputs/gemma3-4b-rag to gs://model-finetune-1/checkpoints/gemma3-4b-rag ...
[GCSSync] Sync complete.
[GCSSync] Syncing /home/yashpareek_workmail/unsloth_finetuning/notebooks/outputs/gemma3-4b-rag to gs://model-finetune-1/check

TrainOutput(global_step=8108, training_loss=2.1408763635199706, metrics={'train_runtime': 27333.9796, 'train_samples_per_second': 1.187, 'train_steps_per_second': 0.297, 'total_flos': 3.1683510571566246e+17, 'train_loss': 2.1408763635199706, 'epoch': 1.0})

In [10]:
from pathlib import Path

save_dir = Path("rag-instruct-gemma-3-finetuned")

save_dir.mkdir(exist_ok=True)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("Saved to", save_dir.resolve())

Saved to /home/yashpareek_workmail/unsloth_finetuning/notebooks/rag-instruct-gemma-3-finetuned


In [ ]:
from unsloth import FastModel
from transformers import AutoTokenizer

base_model_name = MODEL_NAME
adapter_dir = "rag-instruct-gemma-3-finetuned"

base_model, base_tokenizer = FastModel.from_pretrained(
    base_model_name,
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
)

from peft import PeftModel

model = PeftModel.from_pretrained(base_model, adapter_dir)
tokenizer = base_tokenizer

### To get checkpoint to train again

In [ ]:
last_ckpt = get_last_checkpoint(OUTPUT_DIR)
if last_ckpt is not None:
    print(f"Resuming from checkpoint: {last_ckpt}")
    trainer.train(resume_from_checkpoint=last_ckpt)
else:
    print("No checkpoint found. Starting fresh.")
    trainer.train()

print("Saving final adapter + tokenizer...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

## Evaluate Base vs Fine-tuned Model

This section deals with evaluating the performance of a base model against a fine-tuned model using a specific dataset. The evaluation process involves loading both models, running them on the same eval dataset, and comparing their outputs based on predefined metrics.

In [6]:
eval_dataset_demo = eval_dataset.shuffle(seed=SEED).select(range(5))  # small eval for demo
eval_dataset_demo[0]

{'text': "<start_of_turn>user\nYou are a helpful assistant. Use ONLY the provided documents to answer the question.\n\nQUESTION:\nIdentify a statistical approach that explores the structure of a text based on word occurrence patterns instead of meaning.\n\nDOCUMENTS:\nthe University of São Paulo's Institute of Mathematical and Computer Sciences published a paper detailing a study using statistical methods to analyse the relationships of the words in the text. Instead of trying to find the meaning, Amancio's team used complex network modelling to look for connections and clusters of words. By employing concepts such as frequency and intermittence, which measure occurrence and concentration of a term in the text, Amancio was able to discover the manuscript's keywords and create three-dimensional models of the text's structure and word frequencies. Their conclusion was that in 90% of cases, the Voynich systems\n\nof interesting, meaningful pieces of content. By systematically labeling the

In [7]:
import re

USER_BLOCK_RE  = re.compile(r"<start_of_turn>user\s*\n(.*?)<end_of_turn>", re.DOTALL)
MODEL_BLOCK_RE = re.compile(r"<start_of_turn>model\s*\n(.*?)<end_of_turn>", re.DOTALL)

def extract_prompt_and_ref(chat_text: str):
    um = USER_BLOCK_RE.search(chat_text)
    mm = MODEL_BLOCK_RE.search(chat_text)
    if not um or not mm:
        raise ValueError("Could not parse <start_of_turn> blocks from eval text.")
    user_prompt = um.group(1).strip()
    ref_answer  = mm.group(1).strip()
    return user_prompt, ref_answer

# sanity check on example 0
prompt0, ref0 = extract_prompt_and_ref(eval_dataset_demo["text"][0])
print("PROMPT:\n", prompt0)
print("REF:\n", ref0)

PROMPT:
 You are a helpful assistant. Use ONLY the provided documents to answer the question.

QUESTION:
Identify a statistical approach that explores the structure of a text based on word occurrence patterns instead of meaning.

DOCUMENTS:
the University of São Paulo's Institute of Mathematical and Computer Sciences published a paper detailing a study using statistical methods to analyse the relationships of the words in the text. Instead of trying to find the meaning, Amancio's team used complex network modelling to look for connections and clusters of words. By employing concepts such as frequency and intermittence, which measure occurrence and concentration of a term in the text, Amancio was able to discover the manuscript's keywords and create three-dimensional models of the text's structure and word frequencies. Their conclusion was that in 90% of cases, the Voynich systems

of interesting, meaningful pieces of content. By systematically labeling the content of a set of texts, re

In [ ]:
# Inferencing with base model
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="gemma-3",
)
model.to("cuda")
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """You are a helpful assistant. 
        Use ONLY the provided documents to answer the question.
        \n\nQUESTION:\nIdentify a statistical approach that explores the structure of a text based on word occurrence patterns instead of meaning.
        \n\nDOCUMENTS:\nthe University of São Paulo's Institute of Mathematical and Computer Sciences published a paper detailing a study using statistical methods to analyse the relationships of the words in the text. Instead of trying to find the meaning, Amancio's team used complex network modelling to look for connections and clusters of words. By employing concepts such as frequency and intermittence, which measure occurrence and concentration of a term in the text, Amancio was able to discover the manuscript's keywords and create three-dimensional models of the text's structure and word frequencies. Their conclusion was that in 90% of cases, the Voynich systems 
        of interesting, meaningful pieces of content. By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts. Computers are increasingly used in content analysis to automate the labeling (or coding) of documents. Simple computational techniques can provide descriptive data such as word frequencies and document lengths. Machine learning classifiers can greatly increase the number of texts that can be labeled, but the scientific utility of doing so is a matter of debate. Content analysis is best understood as a broad""",
            }
        ],
    }
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Must add for generation
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)
outputs = model.generate(
    **inputs.to("cuda"),
    max_new_tokens=64,
    temperature=0.1,
    top_p=0,
    top_k=64,
)
tokenizer.batch_decode(outputs)

re.search(
    r"<start_of_turn>model\s*\n(.*?)<end_of_turn>",
    tokenizer.batch_decode(outputs)[0],
    re.DOTALL,
).group(1).strip()

'The statistical approach described in the document is complex network modelling. Amancio’s team used this method to look for connections and clusters of words based on their occurrence patterns (frequency and intermittence) rather than focusing on the meaning of the words.'

In [ ]:
# Inferencing with adapted model
from peft import PeftModel

ADAPTER_DIR = "rag-instruct-gemma-3-finetuned"
adapted_model = PeftModel.from_pretrained(model, ADAPTER_DIR)
adapted_outputs = adapted_model.generate(
    **inputs.to("cuda"),
    max_new_tokens=64,
    temperature=0.1,
    top_p=0,
    top_k=64,
)
tokenizer.batch_decode(adapted_outputs)

re.search(
    r"<start_of_turn>model\s*\n(.*?)<end_of_turn>",
    tokenizer.batch_decode(adapted_outputs)[0],
    re.DOTALL,
).group(1).strip()

/home/yashpareek_workmail/unsloth-venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


/home/yashpareek_workmail/unsloth-venv/lib/python3.12/site-packages/peft/peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.language_model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.language_model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.language_model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.language_model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.language_model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.language_model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.language_model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.language_model.layers.0.self_attn.o_proj.lora_B.defaul

'The statistical approach described in the document is complex network modelling. Amancio’s team used this method to look for connections and clusters of words based on their occurrence patterns (frequency and intermittence) rather than focusing on the meaning of the words.'

In [ ]:
if True:
    from unsloth import FastModel

    model, tokenizer = FastModel.from_pretrained(
        model_name=ADAPTER_DIR,
        max_seq_length=2048,
        load_in_4bit=True,
    )

model.to("cuda")
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)

from transformers import TextStreamer

_ = model.generate(
    **inputs.to("cuda"),
    max_new_tokens=100,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
    streamer=TextStreamer(tokenizer, skip_prompt=True),
)

==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Gemma3 does not support SDPA - switching to fast eager.
The statistical approach that explores the structure of a text based on word occurrence patterns is a complex network modelling using concepts like frequency and intermittence. This approach aims to uncover connections and clusters of words in a text by analyzing patterns of word occurrence without focusing on the meaning of the words. For instance, in the Voynich manuscript, Amancio's team used such methods to create three-dimensional models of the text's structure and word frequencies. This technique is notable because it doesn't attempt to


### Load and inference with both models
Try running inference with both models on the same input to see the difference in outputs.

In [ ]:
from unsloth import FastModel
from transformers import TextStreamer

# ---- LOAD BASE (no adapter) ----
base_model, base_tokenizer = FastModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    load_in_4bit=True,
)

# ---- LOAD ADAPTED (adapter dir) ----
adapt_model, adapt_tokenizer = FastModel.from_pretrained(
    model_name=ADAPTER_DIR,
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_16bit=False,
)


# ---- helper: generate output text (same apply_chat_template style) ----
def generate_text(model, tokenizer, messages, max_new_tokens=100):
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True,
    )

    outputs = model.generate(
        **inputs.to("cuda"),
        max_new_tokens=max_new_tokens,
        temperature=1.0,
        top_p=0.95,
        top_k=64,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]


messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """You are a helpful assistant. 
        Use ONLY the provided documents to answer the question.
        \n\nQUESTION:\nIdentify a statistical approach that explores the structure of a text based on word occurrence patterns instead of meaning.
        \n\nDOCUMENTS:\nthe University of São Paulo's Institute of Mathematical and Computer Sciences published a paper detailing a study using statistical methods to analyse the relationships of the words in the text. Instead of trying to find the meaning, Amancio's team used complex network modelling to look for connections and clusters of words. By employing concepts such as frequency and intermittence, which measure occurrence and concentration of a term in the text, Amancio was able to discover the manuscript's keywords and create three-dimensional models of the text's structure and word frequencies. Their conclusion was that in 90% of cases, the Voynich systems 
        of interesting, meaningful pieces of content. By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts. Computers are increasingly used in content analysis to automate the labeling (or coding) of documents. Simple computational techniques can provide descriptive data such as word frequencies and document lengths. Machine learning classifiers can greatly increase the number of texts that can be labeled, but the scientific utility of doing so is a matter of debate. Content analysis is best understood as a broad""",
            }
        ],
    }
]

base_decoded = generate_text(base_model, base_tokenizer, messages, max_new_tokens=100)
adapt_decoded = generate_text(
    adapt_model, adapt_tokenizer, messages, max_new_tokens=100
)


import re


def extract_answer(decoded):
    return (
        re.search(r"<start_of_turn>model\s*\n(.*?)<end_of_turn>", decoded, re.DOTALL)
        .group(1)
        .strip()
    )


base_answer = extract_answer(base_decoded)
adapt_answer = extract_answer(adapt_decoded)

print("\n================ BASE ANSWER ================\n", base_answer)
print("\n=============== ADAPTER ANSWER ==============\n", adapt_answer)
print("\nChanged?", base_answer != adapt_answer)

del base_model
del adapt_model
import torch
import gc


def clear_gpu():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()


clear_gpu()

==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 doe

### Try first 5 examples from eval dataset

In [ ]:
from unsloth import FastModel
from transformers import TextStreamer

# ---- LOAD BASE (no adapter) ----
base_model, base_tokenizer = FastModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=2048,
    load_in_4bit=True,
)

# ---- LOAD ADAPTED (adapter dir) ----
adapt_model, adapt_tokenizer = FastModel.from_pretrained(
    model_name=ADAPTER_DIR,
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_16bit=False,
)


## Metrics
def _normalize(s: str) -> str:
    return " ".join(s.lower().strip().split())


def exact_match(pred: str, ref: str) -> float:
    return float(_normalize(pred) == _normalize(ref))


def _lcs_len(a, b):
    n, m = len(a), len(b)
    dp = [0] * (m + 1)
    for i in range(1, n + 1):
        prev = 0
        for j in range(1, m + 1):
            tmp = dp[j]
            if a[i - 1] == b[j - 1]:
                dp[j] = prev + 1
            else:
                dp[j] = max(dp[j], dp[j - 1])
            prev = tmp
    return dp[m]


def rouge_l_f1(pred: str, ref: str) -> float:
    p = _normalize(pred).split()
    r = _normalize(ref).split()
    if not p or not r:
        return 0.0
    lcs = _lcs_len(p, r)
    prec = lcs / len(p)
    rec = lcs / len(r)
    return (2 * prec * rec / (prec + rec)) if (prec + rec) else 0.0


# Parsing Answer
def extract_answer(decoded):
    m = re.search(r"<start_of_turn>model\s*\n(.*?)<end_of_turn>", decoded, re.DOTALL)
    if m:
        return m.group(1).strip()

    # Fallback 1: if model start exists but no end_of_turn (truncated)
    m2 = re.search(r"<start_of_turn>model\s*\n(.*)$", decoded, re.DOTALL)
    if m2:
        return m2.group(1).strip()

    # Fallback 2: no tags at all -> return decoded as-is
    return decoded.strip()


def generate_text(model, tokenizer, messages, max_new_tokens=100):
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True,
    )

    outputs = model.generate(
        **inputs.to("cuda"),
        max_new_tokens=max_new_tokens,
        temperature=1.0,
        top_p=0.95,
        top_k=64,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]


results = []
for i in range(5):
    chat_text = eval_dataset_demo["text"][i]
    user_prompt, ref = extract_prompt_and_ref(chat_text)

    # build messages exactly like you do (multimodal text wrapper)
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_prompt,
                }
            ],
        }
    ]

    base_decoded = generate_text(
        base_model, base_tokenizer, messages, max_new_tokens=100
    )
    adapt_decoded = generate_text(
        adapt_model, adapt_tokenizer, messages, max_new_tokens=100
    )

    pred_base = extract_answer(base_decoded)
    pred_adpt = extract_answer(adapt_decoded)

    results.append(
        {
            "i": i,
            "em_base": exact_match(pred_base, ref),
            "em_adpt": exact_match(pred_adpt, ref),
            "rl_base": rouge_l_f1(pred_base, ref),
            "rl_adpt": rouge_l_f1(pred_adpt, ref),
            "ref": ref,
            "base": pred_base,
            "adapted": pred_adpt,
        }
    )

# ======================
# PRINT SUMMARY + DETAILS
# ======================
print("\n=== SUMMARY (5 examples) ===")
print(
    "EM   base:",
    sum(r["em_base"] for r in results) / len(results),
    " adapted:",
    sum(r["em_adpt"] for r in results) / len(results),
)
print(
    "RL-F1 base:",
    sum(r["rl_base"] for r in results) / len(results),
    " adapted:",
    sum(r["rl_adpt"] for r in results) / len(results),
)

for r in results:
    print(f"\n--- EX {r['i']} ---")
    print("REF:\n", r["ref"])
    print("\nBASE:\n", r["base"])
    print("\nADAPTED:\n", r["adapted"])
    print(
        f"\nScores: EM base={r['em_base']} adpt={r['em_adpt']} | ROUGE-L base={r['rl_base']:.3f} adpt={r['rl_adpt']:.3f}"
    )

# ======================
# CLEANUP GPU (your style)
# ======================
del base_model
del adapt_model
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.034 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3 doe

ROUGE-L F1 improved from 0.155 → 0.217 (≈ +0.062 absolute, ~+40% relative). That means the adapted model’s answers share more subsequence overlap with the reference answers overall.